In [ ]:
#@title *code:* library imports
import tensorflow.keras.layers as tfkl
import tensorflow.keras as tfk
%tensorflow_version 2.x
#device_name = tf.test.gpu_device_name()
#if device_name != '/device:GPU:0':
#  raise SystemError('GPU device not found')
#print('Found GPU at: {}'.format(device_name))
from tensorflow.keras.models import Model
from tensorflow.keras import layers, losses
import keras.backend as K
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import dill
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/course-IA2/Work-Max/Proyecto')
print(os.getcwd())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/course-IA2/Work-Max/Proyecto


In [ ]:
class lstm_bottleneck(tfkl.Layer):
    def __init__(self, lstm_units, time_steps, **kwargs):
        self.lstm_units = lstm_units
        self.time_steps = time_steps
        self.lstm_layer = tfkl.Bidirectional(tfkl.LSTM(lstm_units,return_sequences=False))
        self.repeat_layer = tfkl.RepeatVector(time_steps)
        super(lstm_bottleneck, self).__init__(**kwargs)
    
    def call(self, inputs):
        # just call the two initialized layers
        return self.repeat_layer(self.lstm_layer(inputs))
    
    def compute_mask(self, inputs, mask=None):
        # return the input_mask directly
        return mask

In [ ]:
latent_dim = 7

class Autoencoder(Model):
  def __init__(self, latent_dim):
    super(Autoencoder, self).__init__()
    self.latent_dim = latent_dim   
    self.encoder = tfk.Sequential()
    self.encoder.add(tfkl.Input(shape=(32, 3)))
    self.encoder.add(tfk.layers.Masking(mask_value=0.))

    self.encoder.add(tfkl.Bidirectional(tfkl.LSTM(units=32,return_sequences=True)))
    self.encoder.add(lstm_bottleneck(lstm_units=self.latent_dim, time_steps=32))

    self.decoder = tfk.Sequential()

    self.decoder.add(tfkl.Bidirectional(tfkl.LSTM(units=32,return_sequences=True)))
    self.decoder.add(tfkl.Bidirectional(tfkl.LSTM(units=3,return_sequences=True)))
    self.decoder.add(tfk.layers.Dense(3,activation='elu'))
    
  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded
  

autoencoder = Autoencoder(latent_dim,)

In [ ]:
autoencoder.build((None,32,3))
autoencoder.load_weights("autoencoder_LSTM_ckpt_weights" ).expect_partial()
autoencoder.encoder.summary()
autoencoder.decoder.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking_1 (Masking)          (None, 32, 3)             0         
_________________________________________________________________
bidirectional_4 (Bidirection (None, 32, 64)            9216      
_________________________________________________________________
lstm_bottleneck_1 (lstm_bott (None, 32, 14)            4032      
Total params: 13,248
Trainable params: 13,248
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_6 (Bidirection (None, 32, 64)            12032     
_________________________________________________________________
bidirectional_7 (Bidirection (None, 32, 6)             1632      
____________________________

In [ ]:
autoencoder.summary()

Model: "autoencoder_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_2 (Sequential)    (None, 32, 14)            13248     
_________________________________________________________________
sequential_3 (Sequential)    (None, 32, 3)             13685     
Total params: 26,933
Trainable params: 26,933
Non-trainable params: 0
_________________________________________________________________


In [ ]:
def mask(arr,mask_value=-8):
  sel=np.diff(arr,axis=0)
  sel=np.sum(sel,axis=1)!=0.
  sel=np.concatenate((sel,[False]))
  return sum(sel)

In [ ]:
def masked_mse(prediction,real_vals,mask,mask_value=-8):
  dif=prediction[:mask]-real_vals[:mask]
  mse = (np.square(dif)).mean(axis=None)
  return mse

In [ ]:
cep = np.load('data/ogle_train_data.npy')
n=int(len(cep)*0.8)
cep=cep[n:]#test_data
pred_cep=autoencoder.predict(cep)

In [ ]:
mask_cep=[mask(s) for s in cep]
mse_cep=[masked_mse(prediction,cep[i],mask_cep[i],0.) for i,prediction in enumerate(pred_cep)]
mse_cep=np.array(mse_cep)
MSE_cep=(mse_cep.mean(),mse_cep.max(),mse_cep.min())
print('Mean MSE for Cep data is: ',MSE_cep[0])
print('Max MSE for Cep data is:',MSE_cep[1])
print('Min MSE for Cep data is:',MSE_cep[2])

Mean MSE for Cep data is:  5819.561664891817
Max MSE for Cep data is: 190481.8768829917
Min MSE for Cep data is: 23.640436702061876


In [ ]:
rrlyr = np.load('data/OGLE_data/rrlyr.npy')
rrly=rrlyr[:2000]
pred_rrlyr=autoencoder.predict(rrlyr)

In [ ]:
mask_rrlyr=[mask(s) for s in rrlyr]
mse_rrlyr=[masked_mse(prediction,rrlyr[i],mask_rrlyr[i],-8.) for i,prediction in enumerate(pred_rrlyr)]
mse_rrlyr=np.array(mse_rrlyr)
MSE_rrlyr=(mse_rrlyr.mean(),mse_rrlyr.max(),mse_rrlyr.min())
print('Mean MSE for Rrlyr data is: ',MSE_rrlyr[0])
print('Max MSE for Rrlyr data is:',MSE_rrlyr[1])
print('Min MSE for Rrlyr data is:',MSE_rrlyr[2])

Mean MSE for Rrlyr data is:  11966346.737544492
Max MSE for Rrlyr data is: 22383903.090269934
Min MSE for Rrlyr data is: 9820725.792896278


In [ ]:
acep = np.load('data/OGLE_data/acep.npy')
acep=acep[:2000]
pred_acep=autoencoder.predict(acep)

In [ ]:
mask_acep=[mask(s) for s in acep]
mse_acep=[masked_mse(prediction,acep[i],mask_acep[i],-8.) for i,prediction in enumerate(pred_acep)]
mse_acep=np.array(mse_acep)
MSE_acep=(mse_acep.mean(),mse_acep.max(),mse_acep.min())
print('Mean MSE for Acep data is: ',MSE_acep[0])
print('Max MSE for Acep data is:',MSE_acep[1])
print('Min MSE for Acep data is:',MSE_acep[2])

Mean MSE for Acep data is:  10483469.925480977
Max MSE for Acep data is: 11124888.48133775
Min MSE for Acep data is: 9914071.185982602


In [ ]:
t2cep = np.load('data/OGLE_data/t2cep.npy')
t2cep=t2cep[:2000]
pred_t2cep=autoencoder.predict(t2cep)

In [ ]:
mask_t2cep=[mask(s) for s in t2cep]
mse_t2cep=[masked_mse(prediction,t2cep[i],mask_t2cep[i],-8.) for i,prediction in enumerate(pred_t2cep)]
mse_t2cep=np.array(mse_t2cep)
MSE_t2cep=(mse_t2cep.mean(),mse_t2cep.max(),mse_t2cep.min())
print('Mean MSE for T2CEP data is: ',MSE_t2cep[0])
print('Max MSE for T2CEP data is:',MSE_t2cep[1])
print('Min MSE for T2CEP data is:',MSE_t2cep[2])

Mean MSE for T2CEP data is:  10483469.925480977
Max MSE for T2CEP data is: 11124888.48133775
Min MSE for T2CEP data is: 9914071.185982602


In [ ]:
ecl = np.load('data/OGLE_data/ecl.npy')
ecl=ecl[:2000]
pred_ecl=autoencoder.predict(ecl)

In [ ]:
mask_ecl=[mask(s) for s in ecl]
mse_ecl=[masked_mse(prediction,ecl[i],mask_ecl[i],-8.) for i,prediction in enumerate(pred_ecl)]
mse_ecl=np.array(mse_ecl)
MSE_ecl=(mse_ecl.mean(),mse_ecl.max(),mse_ecl.min())
print('Mean MSE for Ecl data is: ',MSE_ecl[0])
print('Max MSE for Ecl data is:',MSE_ecl[1])
print('Min MSE for Ecl data is:',MSE_ecl[2])

Mean MSE for Ecl data is:  10483469.925480977
Max MSE for Ecl data is: 11124888.48133775
Min MSE for Ecl data is: 9914071.185982602


In [ ]:
print('CEP',MSE_cep)
print('RRLYR',MSE_rrlyr)
print('ACEP',MSE_acep)
print('ECL',MSE_ecl)
print('T2CEP',MSE_t2cep)

CEP (11809439.445997503, 15695147.678000279, 9820688.413735067)
RRLYR (11966346.737544492, 22383903.090269934, 9820725.792896278)
ACEP (10483469.925480977, 11124888.48133775, 9914071.185982602)
ECL (10483469.925480977, 11124888.48133775, 9914071.185982602)
T2CEP (10483469.925480977, 11124888.48133775, 9914071.185982602)
